In [1]:
# Import 設定
import sys
from pathlib import Path
PROJECT_DIR = Path.cwd().parents[0]
sys.path.insert(0, PROJECT_DIR.joinpath("src").__str__())  # 不能用 pathlib.PosixPath 傳入

# Step 01: Ingestion

In [2]:
from ingestion.file_loaders.goodnotes import DoclingGoodnotesLoader
loader = DoclingGoodnotesLoader()

loader_results = []
for filepath in ["早安日語-孫寅華part11-page01.pdf",]:
    print(filepath)
    loader_results = loader.load(filepath)

Fetching 21 files:   0%|          | 0/21 [00:00<?, ?it/s]

早安日語-孫寅華part11-page01.pdf


2025-08-26 07:59:36,421 - OrtInferSession - WARNING: DirectML is only supported in Windows OS. The current OS is Linux. Use TensorrtExecutionProvider inference by default.
2025-08-26 07:59:36,665 - OrtInferSession - WARNING: DirectML is only supported in Windows OS. The current OS is Linux. Use TensorrtExecutionProvider inference by default.
2025-08-26 07:59:36,682 - OrtInferSession - WARNING: DirectML is only supported in Windows OS. The current OS is Linux. Use TensorrtExecutionProvider inference by default.
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)


In [5]:
loader_results

[LoaderResult(content='文\n\n文型と表現 (lesson \'1\'\'5\'\'1\'\'~\'\'1\'\'5\'\'2\')\n\n」 もうずいぶん雪が積もっていろわね\n\n型と表現 (lesson \'1\'\'5\'\'1\'\'~\'\'1\'\'5\'\'2\')\n\nずいぶん雪が積もっていろわね\n\n２、これじゃ、大雪になりそうだね\n\n## 大雪になりそうだね こねじゃ\n\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}\n\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}\n\n(lesson \'1\'\'5\'\'1\'\'~\'\'1\'\'5\'\'2\')\n\n型と表現\n\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}\n\nだ人は雪が降ても、あまり積もらない人だ\n\n\'4\'.寒くたって、だいじょうぶだよ\n\n\'2\'、これじゃ\',\'大雪になりそうだね\n\nや\n\n→表“樣態，V連用+だ\n\n例：もうすぐ雨が止みそうです\n\nが\n\n自動詞\n\n\'1\'.トラックから荷物を落ちそうです\n\n\'2\'\',\'今にも雨が降りそうです\n\nうめ\n\n\'3\'.もうすぐ梅の花が咲主そうです\n\n\'4\'.そろそろガソりンがなくなりそうです。\n\nた\n\n\'4\'.寒くたって、だいじょうぶだよ\n\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}\n\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}\n\nTable Part 001\n{\n  "col000":{\n    "column_name":"row_name"\n  }\n}\n\nTable Part 001\n{\n  "col000

# Step 2: Chunk

In [6]:
from chunking.docling import DoclingChunkProcessor
chunker = DoclingChunkProcessor()

chunks = []
for loader_result in loader_results:
    chunks += chunker.process(doc=loader_result.doc,
                              metadata=loader_result.metadata,
                              )

### 手動將 Outline 加入 Chunk

In [7]:
for idx in range(len(chunks)):
    chunks[idx].content = "\n".join(chunks[idx].metadata.outlines) + "\n\n" + chunks[idx].content

### 手動調整頁碼

In [11]:
import re
for idx in range(len(chunks)):
    filename = chunks[idx].metadata.file_name
    if filename.startswith("早安日語-孫寅華"):
        regex = re.search(r"早安日語-孫寅華part(\d+)-page(\d+)", filename)
        if regex:
            part_num = int(regex.group(1))
            page_num = int(regex.group(2))
    chunks[idx].metadata.page += (part_num-1)*page_num


In [12]:
len(chunks)

149

In [13]:
print(chunks[0]._raw_chunk)

text="文\n文型と表現 (lesson '1''5''1''~''1''5''2')\n」 もうずいぶん雪が積もっていろわね\n型と表現 (lesson '1''5''1''~''1''5''2')\nずいぶん雪が積もっていろわね\n２、これじゃ、大雪になりそうだね" meta=DocMeta(schema_name='docling_core.transforms.chunker.DocMeta', version='1.0.0', doc_items=[DocItem(self_ref='#/texts/0', parent=RefItem(cref='#/groups/0'), children=[], content_layer=<ContentLayer.BODY: 'body'>, label=<DocItemLabel.TEXT: 'text'>, prov=[ProvenanceItem(page_no=1, bbox=BoundingBox(l=1084.2762044270833, t=127.13716634114587, r=1136.4269205729167, b=80.38716634114587, coord_origin=<CoordOrigin.BOTTOMLEFT: 'BOTTOMLEFT'>), charspan=(0, 1))]), DocItem(self_ref='#/texts/1', parent=RefItem(cref='#/body'), children=[], content_layer=<ContentLayer.BODY: 'body'>, label=<DocItemLabel.TEXT: 'text'>, prov=[ProvenanceItem(page_no=2, bbox=BoundingBox(l=653.3164876302084, t=134.97176106770837, r=1065.17333984375, b=78.4193115234375, coord_origin=<CoordOrigin.BOTTOMLEFT: 'BOTTOMLEFT'>), charspan=(0, 36))]), DocItem(self_ref='#/texts/2', parent=RefI

/tmp/ipykernel_53058/3635813002.py:1: UserWarning: _raw_chunk 為備份欄位，僅應在 chunking 階段或 debugging 時使用
  print(chunks[0]._raw_chunk)


# Step 3: Embedding

In [14]:
import os
from cache.redis import RedisCacheHandler
from embedding.openai_embed import OpenAIEmbeddingModel


embedder = OpenAIEmbeddingModel(api_key=os.getenv("OPEN_AI_API"),
                                model_name="text-embedding-3-small",
                                memory_cache=RedisCacheHandler(host=os.getenv("MY_REDIS_HOST"),
                                                               port=os.getenv("MY_REDIS_PORT"),
                                                               password=os.getenv("MY_REDIS_PASSWORD"),
                                                               ),
                                )
for idx in range(len(chunks)):
    vector = embedder.encode(chunks[idx].content)
    chunks[idx].embedding = vector

# Step 04: Insert

## Vector-based

In [15]:
from infra.stores.pgvector import PGVectorStore

vec_store = PGVectorStore(host=os.getenv("MY_POSTGRE_HOST"),
                          port=os.getenv("MY_POSTGRE_PORT"),
                          dbname=os.getenv("MY_POSTGRE_DB_NAME"),
                          schema="Japanese-Learning",
                          user=os.getenv("MY_POSTGRE_USERNAME"),
                          password=os.getenv("MY_POSTGRE_PASSWORD"),
                          )

for chunk in chunks:
    vec_store.insert(chunk)

## Lexical-based

In [16]:
from infra.stores.elasticsearch import ElasticsearchBM25Store

lex_store = ElasticsearchBM25Store(host=os.getenv("MY_ELASTIC_HOST"),
                                   port=os.getenv("MY_ELASTIC_PORT"),
                                   index_name="japanese-learning",
                                   username=os.getenv("MY_ELASTIC_USERNAME"),
                                   password=os.getenv("MY_ELASTIC_PASSWORD"),
                                   )

for chunk in chunks:
    lex_store.insert(chunk)

# Step 05: Retrieve

## Vector-based

In [18]:
import os

from cache.redis import RedisCacheHandler
from embedding.openai_embed import OpenAIEmbeddingModel
from infra.stores.pgvector import PGVectorStore

embedder = OpenAIEmbeddingModel(api_key=os.getenv("OPEN_AI_API"),
                                model_name="text-embedding-3-small",
                                memory_cache=RedisCacheHandler(host=os.getenv("MY_REDIS_HOST"),
                                                               port=os.getenv("MY_REDIS_PORT"),
                                                               password=os.getenv("MY_REDIS_PASSWORD"),
                                                               ),
                                )
query_vector = embedder.encode("自動詞與他動詞的判斷邏輯")

vec_store = PGVectorStore(host=os.getenv("MY_POSTGRE_HOST"),
                          port=os.getenv("MY_POSTGRE_PORT"),
                          dbname=os.getenv("MY_POSTGRE_DB_NAME"),
                          schema="Japanese-Learning",
                          user=os.getenv("MY_POSTGRE_USERNAME"),
                          password=os.getenv("MY_POSTGRE_PASSWORD"),
                          )
results = vec_store.search(query_vector)

In [19]:
[_.chunk.content for _ in results]

['（文型）補助動詞「他動詞てある」：表動作完成後，狀態的持續、停留在那裡\n（文型）補助動詞「～てあげる」：表幫忙做某事、施予恩惠\n\n降る\nその後\nあと',
 "（文型）補助動詞「他動詞てある」：表動作完成後，狀態的持續、停留在那裡\n（文型）補助動詞「～てあげる」：表幫忙做某事、施予恩惠\n\n'-'\nふゆ にほん 日本の冬の気候は、\nせいてん かんそう 乾燥した晴天の日が続 持續\nインターチェンジ\n(lesson'1''5''4''~''1''5'",
 '（文型）補助動詞「他動詞てある」：表動作完成後，狀態的持續、停留在那裡\n（文型）補助動詞「～てあげる」：表幫忙做某事、施予恩惠\n\nにほんふゆ 日本の冬の気候は、',
 "（文型）補助動詞「他動詞てある」：表動作完成後，狀態的持續、停留在那裡\n（文型）補助動詞「～てあげる」：表幫忙做某事、施予恩惠\n\nにほんかいがわ\nまいにち\n'*'\n<も\nしかし、日本海側では、毎日どんよりとした曇\n副詞語尾と\n(時常見到聲，擬態語+と的用\n此時'{'と_表動作或狀態的樣\nEx：い)ろい'3'と考えてみる\n意外と多い\nい)が\n'5'）\nちが たいへいようがわ 違う。太平洋側では\nにち くも 日どんよりとした曇",
 "（文型）補助動詞「他動詞てある」：表動作完成後，狀態的持續、停留在那裡\n（文型）補助動詞「～てあげる」：表幫忙做某事、施予恩惠\n\n(時常見到擬聲.擬態語+と的用法)\n此時と表動作或狀態的樣子\nEx：いろい'3'と考えみる\n意外と多い\nせいてん つ かんそう 乾燥した晴天の日が続 持續\nつ にほん りや雪の日が続く。日本\nちほう たい 帯だ。これの地方は\nは、シア吹いて\nにほんかい じようくう とお と い日本海の上空を通る時\nにほんれとう ちゆうおう 日本列島の中央あ高\nすいじようき かた ゆき 水蒸気が固まり雪になる\nせいてん つ かんそう にほんかいがわ ま 持續 つブ にほんかいがわほくりくちほうとうほくちほう りや雪の日続。日本海側の北陸地方や東北地 ちほう たい ゆきに だ帯だ。これらの地方では、雪が２メートルから'5'メ Recall：進階 ふ は、シベリアから吹いて来る冷たい季節風のためだ。 つめ きせつふう 表緣故 

## Lexical-based

In [20]:
import os

from infra.stores.elasticsearch import ElasticsearchBM25Store

lex_store = ElasticsearchBM25Store(host=os.getenv("MY_ELASTIC_HOST"),
                                   port=os.getenv("MY_ELASTIC_PORT"),
                                   index_name="japanese-learning",
                                   username=os.getenv("MY_ELASTIC_USERNAME"),
                                   password=os.getenv("MY_ELASTIC_PASSWORD"),
                                   )
results = lex_store.search("自動詞與他動詞的判斷邏輯")

In [21]:
[_.chunk.content for _ in results]

["「好像」總整理（そうだ・らしい・ようだ・みたい）\n\n'['ある']'ない\nなそう\n食べなそう\n食べたくない→食べたくなさそう\n食べない\n容赦ない\n容赦しない\n容赦なそう\n容赦しなそう\n面白くない\n日本人也不全和文法相同\n→面自くそう\nうまらだい\n一っまらばそう\n只有動詞的い変化才會用「なそう」(除了ある)\n最有u，结尾的A啦，是他又不是否定\n但現實生活中",
 '（文型）補助動詞「他動詞てある」：表動作完成後，狀態的持續、停留在那裡\n（文型）補助動詞「～てあげる」：表幫忙做某事、施予恩惠\n\n降る\nその後\nあと',
 '（文型）補助動詞「他動詞てある」：表動作完成後，狀態的持續、停留在那裡\n（文型）補助動詞「～てあげる」：表幫忙做某事、施予恩惠\n\nにほんふゆ 日本の冬の気候は、',
 '（文型）補助動詞「他動詞てある」：表動作完成後，狀態的持續、停留在那裡\n（文型）補助動詞「～てあげる」：表幫忙做某事、施予恩惠\n\nきせつふう\nあたた\nこの季節風は、温か\nす かぜ 気を吸い込んだ風が、\nきゆう Ω 空気が急冷され、\nふ',
 "（文型）補助動詞「他動詞てある」：表動作完成後，狀態的持續、停留在那裡\n（文型）補助動詞「～てあげる」：表幫忙做某事、施予恩惠\n\n'-'\nふゆ にほん 日本の冬の気候は、\nせいてん かんそう 乾燥した晴天の日が続 持續\nインターチェンジ\n(lesson'1''5''4''~''1''5'"]